In [30]:
%matplotlib widget

from datetime import *
from dateutil import tz

def datenum_to_pytime(matlab_datenum):
    python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    return python_datetime.strftime('%Y-%m-%d %H:%M:%S')
    
matlab_datenum = 736054.8750082987
print(datenum_to_pytime(matlab_datenum))

2015-03-31 21:00:00


In [2]:
#rootgrp.groups['000100'].variables['IntegrationMidPointTime']:

'''
736055.0135948148 - 2015-04-01 00:19:34.592000 - 219
736055.0136063888 - 2015-04-01 00:19:35.591994 - 220
736055.013617963 - 2015-04-01 00:19:36.591999 - 221
736055.013629537 - 2015-04-01 00:19:37.591993 - 222
'''

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from netCDF4 import Dataset

rootgrp = Dataset("raw/L1B/2017-11-26-H06-DDMs.nc", "r", format="NETCDF4")
metagrp = Dataset("raw/L1B/2017-11-26-H06-metadata.nc", "r", format="NETCDF4")

fig = plt.figure(figsize=(10, 4))

group = '000005'
start = 525;

datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][start]
print(datenum_to_pytime(float(datenum)))
lat = metagrp.groups[group].variables['SpecularPointLat'][start]
lon = metagrp.groups[group].variables['SpecularPointLon'][start]
print('Lat: ' + str(lat) + ' Lon: ' + str(lon))

ddm = rootgrp.groups[group].variables['DDM'][start].data
datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][start]
lat = metagrp.groups[group].variables['SpecularPointLat'][start]
lon = metagrp.groups[group].variables['SpecularPointLon'][start]
string = str(datenum_to_pytime(float(datenum))) + ' Lat: ' + "{0:.2f}".format(lat) + ' Lon: ' + "{0:.2f}".format(lon)

#im = plt.imshow(ddm, animated=True, extent=(0,127,-10,10))

im = plt.imshow(ddm, cmap='viridis', extent=(0,127,-10,9))
t = plt.text(51, 7, string, {'color': 'w', 'fontsize': 12})
    
plt.xlim(50, 100)
plt.ylim(-10, 9)

def animate(i):
    ddm = rootgrp.groups[group].variables['DDM'][i+start].data
    datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][i]
    lat = metagrp.groups[group].variables['SpecularPointLat'][i]
    lon = metagrp.groups[group].variables['SpecularPointLon'][i]
    string = str(datenum_to_pytime(float(datenum))) + ' Lat: ' + "{0:.2f}".format(lat) + ' Lon: ' + "{0:.2f}".format(lon)
    t.set_text(string)
    im.set_array(ddm)
    return im, t

ani = animation.FuncAnimation(fig, animate, frames=100, blit=True, interval=100)
#Play video
HTML(ani.to_html5_video())

FigureCanvasNbAgg()

2017-11-26 03:25:20
Lat: -50.85285 Lon: -19.16286


In [8]:
import numpy
from matplotlib.patches import Circle
from netCDF4 import Dataset

rootgrp = Dataset("raw/2015-04-01-H00-ddm.nc", "r", format="NETCDF4")
metagrp = Dataset("raw/2015-04-01-H00-metadata.nc", "r", format="NETCDF4")
group = '000095'
start = 525;

ddm_1 = numpy.array(rootgrp.groups[group].variables['DDM'][start].data)
ddm_1 = ddm_1/65535
fig_1 = plt.figure(figsize=(10, 4))
im_1 = plt.imshow(ddm_1, cmap='viridis', extent=(0,127,-10,9))

ddm_sum = ddm_1
n = 25
for i in range(n):
    ddm_i = numpy.array(rootgrp.groups[group].variables['DDM'][start-i].data)
    ddm_i = ddm_i/65535
    ddm_sum = (ddm_sum + ddm_i)/2
    
fig_sum = plt.figure(figsize=(10, 4))
im_sum = plt.imshow(ddm_sum, cmap='viridis', extent=(0,127,-10,9))    

ddm_3 = ddm_1 - ddm_sum*0.3
fig_3 = plt.figure(figsize=(10, 4))
im_3 = plt.imshow(ddm_3, cmap='viridis', extent=(0,127,-10,9))

ddm_4 = ddm_3
threshold = 0.78
for row_i, row in enumerate(ddm_4):
    for col_i, col in enumerate(row):
        #ddm_4[row_i][col_i] = 0
        if(col_i >= 60 and col_i <= 80):
            if(ddm_4[row_i][col_i] >= threshold*60/col_i*1.03):
                ddm_4[row_i][col_i] = 1
            else:
                ddm_4[row_i][col_i] = 0
        else:
            ddm_4[row_i][col_i] = 0

fig_4 = plt.figure(figsize=(10, 4))
im_4 = plt.imshow(ddm_4, cmap='viridis', extent=(0,127,-10,9))
        
from skimage.measure import label, regionprops
import matplotlib.patches as mpatches

all_labels = label(ddm_4)
fig_6,ax_6 = plt.subplots(1,figsize=(10, 4))
ax_6.imshow(ddm_1, cmap='viridis')
for region in regionprops(all_labels):
    # draw rectangle around segmented coins
    minr, minc, maxr, maxc = region.bbox
    l = 1
    rect = mpatches.Rectangle((minc-l, minr-l), maxc - minc + 2*l-1, maxr - minr + 2*l - 1, fill=False, edgecolor='red', linewidth=2)
    ax_6.add_patch(rect)


/home/woowapdabug/projects/thesis/python/lib/python3.6/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [ ]:
print(rootgrp.groups[group].variables['DDM'])

In [ ]:
import numpy
import os
from netCDF4 import Dataset

#rootgrp = Dataset("raw/2015-04-01-H00-ddm.nc", "r", format="NETCDF4")
filename = os.path.join(os.environ['TDS_ROOT'], 'raw/L1B/2017-11-11-H18-DDMs.nc')
rootgrp = Dataset(filename, "r", format="NETCDF4")

#import cv2
#ddm_sum = cv2.blur(ddm_sum,(2,1))
#ddm_sum = normalize(ddm_sum)

group = '000028'

def normalize(mat):
    return (mat - numpy.min(mat))/(numpy.max(mat)-numpy.min(mat))

start = 664

ddm_1 = numpy.array(rootgrp.groups[group].variables['DDM'][start].data)
ddm_1 = normalize(ddm_1)

ddm_sum = numpy.array(rootgrp.groups[group].variables['DDM'][start-1].data)
n = 100
for i in range(n):
    ddm_i = numpy.array(rootgrp.groups[group].variables['DDM'][start-i].data)
    ddm_i = normalize(ddm_i)
    ddm_sum = 0.5*ddm_sum + 0.5*ddm_i

ddm_3 = ddm_1 - 0.8*ddm_sum
if (numpy.min(ddm_3) < 0):
    ddm_3 = ddm_3 - numpy.min(ddm_3)
#ddm_3 = normalize(ddm_3)

ddm_4 = ddm_3*1
cut_threshold = 0.5
for row_i, row in enumerate(ddm_4):
    for col_i, col in enumerate(row):
        if(ddm_sum[row_i][col_i]<=cut_threshold):
            ddm_4[row_i][col_i] = 0
            numpy.min(ddm_3)

ddm_5 = ddm_4*1
threshold = 0.75
for row_i, row in enumerate(ddm_4):
    for col_i, col in enumerate(row):
        if(col_i >= 60 and col_i <= 80):
            if(ddm_5[row_i][col_i] >= threshold*60/col_i):
                ddm_5[row_i][col_i] = 1
            else:
                ddm_5[row_i][col_i] = 0
        else:
            ddm_5[row_i][col_i] = 0

            
fig_1 = plt.figure(figsize=(10, 4))
im_1 = plt.imshow(ddm_1, cmap='viridis', extent=(0,127,-10,9))

fig_sum = plt.figure(figsize=(10, 4))
im_sum = plt.imshow(ddm_sum, cmap='viridis', extent=(0,127,-10,9))    

fig_3 = plt.figure(figsize=(10, 4))
im_3 = plt.imshow(ddm_3, cmap='viridis', extent=(0,127,-10,9))

fig_4 = plt.figure(figsize=(10, 4))
im_4 = plt.imshow(ddm_4, cmap='viridis', extent=(0,127,-10,9))

fig_5 = plt.figure(figsize=(10, 4))
im_5 = plt.imshow(ddm_5, cmap='viridis', extent=(0,127,-10,9))

from skimage.measure import label, regionprops
import matplotlib.patches as mpatches

all_labels = label(ddm_5)
fig_6,ax_6 = plt.subplots(1,figsize=(10, 4))
ax_6.imshow(ddm_1, cmap='viridis')
for region in regionprops(all_labels):
    # draw rectangle around segmented coins
    minr, minc, maxr, maxc = region.bbox
    l = 1.5
    rect = mpatches.Rectangle((minc-l, minr-l), maxc - minc + 2*l-1, maxr - minr + 2*l - 1, fill=False, edgecolor='red', linewidth=2)
    ax_6.add_patch(rect)

In [ ]:
from skimage.measure import label, regionprops
import matplotlib.patches as mpatches
import numpy
import os
from netCDF4 import Dataset

def normalize(mat):
    return (mat - numpy.min(mat))/(numpy.max(mat)-numpy.min(mat))

#rootgrp = Dataset("raw/2015-04-01-H00-ddm.nc", "r", format="NETCDF4")
#group = '000095'

filename = os.path.join(os.environ['TDS_ROOT'], 'raw/L1B/2017-11-11-H18-DDMs.nc')
rootgrp = Dataset(filename, "r", format="NETCDF4")

group = '000028'
#group = '000095'

start =660

for index in range(start,start+10):

    ddm_1 = numpy.array(rootgrp.groups[group].variables['DDM'][index].data)
    ddm_1 = normalize(ddm_1)

    ddm_sum = numpy.array(rootgrp.groups[group].variables['DDM'][index-1].data)
    n = 20
    for i in range(n):
        ddm_i = numpy.array(rootgrp.groups[group].variables['DDM'][index-i].data)
        ddm_i = normalize(ddm_i)
        ddm_sum = 0.5*ddm_sum + 0.5*ddm_i

    ddm_3 = ddm_1 - 0.8*ddm_sum
    if (numpy.min(ddm_3) < 0):
        ddm_3 = ddm_3 - numpy.min(ddm_3)
    #ddm_3 = normalize(ddm_3)

    ddm_4 = ddm_3*1
    cut_threshold = 0.5
    for row_i, row in enumerate(ddm_4):
        for col_i, col in enumerate(row):
            if(ddm_sum[row_i][col_i]<=cut_threshold):
                ddm_4[row_i][col_i] = 0
                numpy.min(ddm_3)

    ddm_5 = ddm_4*1
    threshold = 0.55
    found = False
    for row_i, row in enumerate(ddm_4):
        for col_i, col in enumerate(row):
            if(col_i >= 60 and col_i <= 80):
                if(ddm_5[row_i][col_i] >= threshold*60/col_i):
                    ddm_5[row_i][col_i] = 1
                    found = True
                else:
                    ddm_5[row_i][col_i] = 0
            else:

                ddm_5[row_i][col_i] = 0

    if (found == True):   
        fig_1 = plt.figure(figsize=(10, 4))
        im_1 = plt.imshow(ddm_1, cmap='viridis', extent=(0,127,-10,9))

        fig_sum = plt.figure(figsize=(10, 4))
        im_sum = plt.imshow(ddm_sum, cmap='viridis', extent=(0,127,-10,9))    

        fig_3 = plt.figure(figsize=(10, 4))
        im_3 = plt.imshow(ddm_3, cmap='viridis', extent=(0,127,-10,9))

        fig_4 = plt.figure(figsize=(10, 4))
        im_4 = plt.imshow(ddm_4, cmap='viridis', extent=(0,127,-10,9))

        fig_5 = plt.figure(figsize=(10, 4))
        im_5 = plt.imshow(ddm_5, cmap='viridis', extent=(0,127,-10,9))
        

        all_labels = label(ddm_5)
        fig_6,ax_6 = plt.subplots(1,figsize=(10, 4))
        ax_6.imshow(ddm_1, cmap='viridis')
        for region in regionprops(all_labels):
            # draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            l = 1.5
            rect = mpatches.Rectangle((minc-l, minr-l), maxc - minc + 2*l-1, maxr - minr + 2*l - 1, fill=False, edgecolor='red', linewidth=2)
            ax_6.add_patch(rect)
        

In [ ]:
import numpy

#import cv2
#ddm_sum = cv2.blur(ddm_sum,(2,1))
#ddm_sum = normalize(ddm_sum)

group = '000095'

def normalize(mat):
    return (mat - numpy.min(mat))/(numpy.max(mat)-numpy.min(mat))

start = 525
int_start = 1
n = 200

ddm_1 = numpy.array(rootgrp.groups[group].variables['DDM'][start].data)
ddm_1 = normalize(ddm_1)

ddm_sum_0 = numpy.array(rootgrp.groups[group].variables['DDM'][start-int_start].data)
ddm_sum_0 = normalize(ddm_sum)
for i in range(n):
    ddm_i = numpy.array(rootgrp.groups[group].variables['DDM'][start-i-int_start].data)
    ddm_i = normalize(ddm_i)
    for row_i, row in enumerate(ddm_1):
        for col_i, col in enumerate(row):
            val_i = ddm_i[row_i][col_i]
            val_sum = ddm_sum_0[row_i][col_i]
            if(val_sum < val_i):
                ddm_sum_0[row_i][col_i] = 0.8*val_sum + 0.2*val_i
            else:
                 ddm_sum_0[row_i][col_i] = 0.2*val_sum + 0.8*val_i
                    
ddm_sum = ddm_sum_0*1
for i in range(n):
    ddm_i = numpy.array(rootgrp.groups[group].variables['DDM'][start-i-int_start].data)
    ddm_i = normalize(ddm_i)
    for row_i, row in enumerate(ddm_1):
        for col_i, col in enumerate(row):
            val_i = ddm_i[row_i][col_i]
            val_sum = ddm_sum[row_i][col_i]
            alpha = 0.7
            ddm_sum[row_i][col_i] = val_sum + alpha*(val_i-val_sum)


ddm_3 = ddm_1 - ddm_sum
if (numpy.min(ddm_3) < 0):
    ddm_3 = ddm_3 - numpy.min(ddm_3)

#import cv2
#ddm_sum = cv2.blur(ddm_sum,(2,2))
#ddm_sum = normalize(ddm_sum)

ddm_4 = ddm_3*1
cut_threshold = 0.3
for row_i, row in enumerate(ddm_4):
    for col_i, col in enumerate(row):
        if(col_i >= 60 and col_i <= 85):
            if(ddm_sum[row_i][col_i]<=cut_threshold):
                ddm_4[row_i][col_i] = 0
        else:
            ddm_4[row_i][col_i] = 0

ddm_5 = ddm_4*1
threshold = 0.55
for row_i, row in enumerate(ddm_4):
    for col_i, col in enumerate(row):
        if(col_i >= 60 and col_i <= 85):
            if(ddm_5[row_i][col_i] >= threshold):
                ddm_5[row_i][col_i] = 1
            else:
                ddm_5[row_i][col_i] = 0
        else:
            ddm_5[row_i][col_i] = 0

                
#GRADEINT SEARCH      
fig_1 = plt.figure(figsize=(10, 4))
im_1 = plt.imshow(ddm_1, cmap='viridis', extent=(0,127,-10,9))

#fig_sum_0 = plt.figure(figsize=(10, 4))
#im_sum_0 = plt.imshow(ddm_sum_0, cmap='viridis', extent=(0,127,-10,9))    

fig_sum = plt.figure(figsize=(10, 4))
im_sum = plt.imshow(ddm_sum, cmap='viridis', extent=(0,127,-10,9))    

fig_3 = plt.figure(figsize=(10, 4))
im_3 = plt.imshow(ddm_3, cmap='viridis', extent=(0,127,-10,9))

fig_4 = plt.figure(figsize=(10, 4))
im_4 = plt.imshow(ddm_4, cmap='viridis', extent=(0,127,-10,9))

fig_5 = plt.figure(figsize=(10, 4))
im_5 = plt.imshow(ddm_5, cmap='viridis', extent=(0,127,-10,9)) 

from skimage.measure import label, regionprops
import matplotlib.patches as mpatches

all_labels = label(ddm_5)
fig_6,ax_6 = plt.subplots(1,figsize=(10, 4))
ax_6.imshow(ddm_1, cmap='viridis')
for region in regionprops(all_labels):
    # draw rectangle around segmented coins
    minr, minc, maxr, maxc = region.bbox
    l = 2
    rect = mpatches.Rectangle((minc-l, minr-l), maxc - minc + 2*l-1, maxr - minr + 2*l - 1, fill=False, edgecolor='red', linewidth=2)
    ax_6.add_patch(rect)

In [ ]:
from netCDF4 import Dataset

rootgrp = Dataset("raw/2015-04-01-H00-ddm.nc", "r", format="NETCDF4")
metagrp = Dataset("raw/2015-04-01-H00-metadata.nc", "r", format="NETCDF4")

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

group = '000095'

lat = 46
lon = -46
fig = plt.figure(figsize=(8, 8))

m = Basemap(projection='merc',llcrnrlat=-80,urcrnrlat=80,\
            llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')
# draw parallels and meridians.
m.drawparallels(np.arange(-90.,91.,30.))
m.drawmeridians(np.arange(-180.,181.,60.))
m.drawmapboundary(fill_color='aqua')


from matplotlib.font_manager import FontProperties

font = FontProperties()
font.set_weight('bold')

start = 0

lat = metagrp.groups[group].variables['SpecularPointLat'][start].data
lon = metagrp.groups[group].variables['SpecularPointLon'][start].data
x, y = m(lon, lat)
point = m.plot(x, y, 'ro', markersize=5)[0]

datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][start]
print(datenum_to_pytime(float(datenum)))
lat = metagrp.groups[group].variables['SpecularPointLat'][start]
lon = metagrp.groups[group].variables['SpecularPointLon'][start]
print('Lat: ' + str(lat) + ' Lon: ' + str(lon))
t = plt.text(51, 7, string, {'color': 'w', 'fontsize': 12})
    
def init():
    point.set_data([], [])
    return point,

# animation function.  This is called sequentially
def animate(i):
    lat = metagrp.groups[group].variables['SpecularPointLat'][start + 5*i].data
    lon = metagrp.groups[group].variables['SpecularPointLon'][start + 5*i].data
    x, y = m(lon, lat)
    point.set_data(x, y)
    
    datenum = rootgrp.groups[group].variables['IntegrationMidPointTime'][i]
    lat = metagrp.groups[group].variables['SpecularPointLat'][i]
    lon = metagrp.groups[group].variables['SpecularPointLon'][i]
    string = str(datenum_to_pytime(float(datenum))) + ' Lat: ' + "{0:.2f}".format(lat) + ' Lon: ' + "{0:.2f}".format(lon)
    t.set_text(string)
    
    return point,t

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=100, blit=True)
HTML(anim.to_html5_video())

In [89]:
#rootgrp.groups['000100'].variables['IntegrationMidPointTime']:

'''
736055.0135948148 - 2015-04-01 00:19:34.592000 - 219
736055.0136063888 - 2015-04-01 00:19:35.591994 - 220
736055.013617963 - 2015-04-01 00:19:36.591999 - 221
736055.013629537 - 2015-04-01 00:19:37.591993 - 222
'''

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from netCDF4 import Dataset

rootgrp = Dataset("raw/L1B/2018-03-30-H06-DDMs.nc", "r", format="NETCDF4")
metagrp = Dataset("raw/L1B/2018-03-30-H06-metadata.nc", "r", format="NETCDF4")
l2 = Dataset("L2_FDI.nc", "r", format="NETCDF4")

In [9]:
print(g) for g in l2.groups

SyntaxError: invalid syntax (<ipython-input-9-b8041875f483>, line 1)

In [26]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from netCDF4 import Dataset

rootgrp = Dataset("raw/L1B/2015-04-01-H00-DDMs.nc", "r", format="NETCDF4")
metagrp = Dataset("raw/L1B/2015-04-01-H00-metadata.nc", "r", format="NETCDF4")

In [8]:
l2 = Dataset("L2_FDI.nc", "r", format="NETCDF4")

In [25]:
wind = dict(zip(l2.variables['IntegrationMidPointTime'][:].data, l2.variables['WindSpeed'][:].data))

In [86]:
mean = 0
for index in range(515-30,515+5):
    key_time = float(rootgrp.groups['000095'].variables['IntegrationMidPointTime'][515].data)
    datenum_to_pytime(key_time)
    mean += wind[key_time]
mean /= 35
print(mean)

14.06
